<a href="https://colab.research.google.com/github/nipun22325/ImageSuperResolutionCV/blob/main/mainFolderNew/SRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/yjn870/SRCNN-pytorch

fatal: destination path 'SRCNN-pytorch' already exists and is not an empty directory.


In [4]:
import tensorflow_datasets as tfds
dataset, info = tfds.load("div2k/bicubic_x4", as_supervised=True, with_info=True)
train_dataset = dataset["train"]
val_dataset = dataset["validation"]


In [5]:
pip install torch torchvision numpy matplotlib tensorflow_datasets


In [6]:
cd SRCNN-pytorch


/content/SRCNN-pytorch


In [7]:
cat models.py

from torch import nn


class SRCNN(nn.Module):
    def __init__(self, num_channels=1):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=9 // 2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=5 // 2)
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding=5 // 2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x


In [8]:

import torch
from models import SRCNN

model = SRCNN()
model.load_state_dict(torch.load('../srcnn_x4.pth'))


<All keys matched successfully>

In [13]:
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.2 MB/s eta 0:00:00


In [19]:
import tensorflow_datasets as tfds
import torch
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import lpips

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

loss_fn_alex = lpips.LPIPS(net='alex').to(device)

metrics = {'psnr': [], 'ssim': [], 'lpips': []}

for idx, (lr_image, hr_image) in enumerate(val_dataset.take(10)):  #Adjust the number of images to test
    lr_image = np.array(lr_image) / 255.0  #Normalize to [0, 1]
    hr_image = np.array(hr_image) / 255.0

    #grayscale as SRCNN takes 1 input channel not 3
    lr_image_gray = cv2.cvtColor((lr_image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY) / 255.0
    hr_image_gray = cv2.cvtColor((hr_image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY) / 255.0

    lr_image_tensor = torch.from_numpy(lr_image_gray).float().unsqueeze(0).unsqueeze(0).to(device)  # Add channel dimension
    hr_image_tensor = torch.from_numpy(hr_image_gray).float().unsqueeze(0).unsqueeze(0).to(device)  # Add channel dimension

    with torch.no_grad():
        output = model(lr_image_tensor).data.squeeze().float().cpu().clamp_(0, 1).numpy()

    output_uint8 = (output * 255.0).round().astype(np.uint8)

    output_resized = cv2.resize(output_uint8, (hr_image_gray.shape[1], hr_image_gray.shape[0]))

    print(f"HR shape: {hr_image_gray.shape}, Output shape: {output_resized.shape}")

    hr_image_uint8 = (hr_image_gray * 255).astype(np.uint8)

    #PSNR
    psnr_value = psnr(hr_image_uint8, output_resized, data_range=255)  # Specify data range
    metrics['psnr'].append(psnr_value)

    #SSIM
    ssim_value = ssim(hr_image_gray, output_resized / 255.0, multichannel=False, data_range=1)  # Specify data range
    metrics['ssim'].append(ssim_value)

    #LPIPS
    output_tensor = torch.from_numpy(output_resized).float().unsqueeze(0).unsqueeze(0).to(device)  # Add channel dimension
    lpips_value = loss_fn_alex(output_tensor, hr_image_tensor).item()
    metrics['lpips'].append(lpips_value)

    print(f"Image {idx+1}: PSNR: {psnr_value:.4f}, SSIM: {ssim_value:.4f}, LPIPS: {lpips_value:.4f}")

if metrics['psnr']:
    avg_psnr = sum(metrics['psnr']) / len(metrics['psnr'])
    avg_ssim = sum(metrics['ssim']) / len(metrics['ssim'])
    avg_lpips = sum(metrics['lpips']) / len(metrics['lpips'])

    print(f"\nAverage PSNR: {avg_psnr:.4f}")
    print(f"Average SSIM: {avg_ssim:.4f}")
    print(f"Average LPIPS: {avg_lpips:.4f}")
else:
    print("No HR images found for comparison.")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
HR shape: (2040, 1668), Output shape: (2040, 1668)
Image 1: PSNR: 17.5484, SSIM: 0.3969, LPIPS: 0.8494
HR shape: (1572, 2040), Output shape: (1572, 2040)
Image 2: PSNR: 18.1456, SSIM: 0.4167, LPIPS: 0.8716
HR shape: (1200, 2040), Output shape: (1200, 2040)
Image 3: PSNR: 18.5409, SSIM: 0.4121, LPIPS: 0.8141
HR shape: (1536, 2040), Output shape: (1536, 2040)
Image 4: PSNR: 22.6395, SSIM: 0.7761, LPIPS: 0.8784
HR shape: (1356, 2040), Output shape: (1356, 2040)
Image 5: PSNR: 22.0861, SSIM: 0.4629, LPIPS: 0.9179
HR shape: (1356, 2040), Output shape: (1356, 2040)
Image 6: PSNR: 18.3707, SSIM: 0.3654, LPIPS: 0.8535
HR shape: (1740, 2040), Output shape: (1740, 2040)
Image 7: PSNR: 18.8302, SSIM: 0.5071, LPIPS: 0.8477
HR shape: (1356, 2040), Output shape: (1356, 2040)
Image 8: PSNR: 21.1263, SSIM: 0.2460, LPIPS: 0.9182
HR shape: (1356,